In [1]:
import socket
import json

# Define the database file name
DB_FILE = "dns_database.json"

# Create a UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind the socket to port 53533
sock.bind(('0.0.0.0', 53533))

# Load the database from the file
try:
    with open(DB_FILE) as f:
        database = json.load(f)
except FileNotFoundError:
    database = {}

# Handle incoming requests
while True:
    data, addr = sock.recvfrom(1024)

    # Parse the request
    request = data.decode()
    request_type, request_name = request.split()

    # Handle registration requests
    if request_type == "REGISTER":
        # Parse the request data
        request_data = addr[0] + " " + request_name
        hostname, ip_address = request_name, addr[0]

        # Add the record to the database
        database[hostname] = ip_address

        # Save the database to the file
        with open(DB_FILE, "w") as f:
            json.dump(database, f)

        # Send a response with code 201
        response = "201 Created"
        sock.sendto(response.encode(), addr)

    # Handle DNS queries
    elif request_type == "QUERY":
        # Check if the record exists in the database
        if request_name in database:
            # Send a response with the IP address and TTL
            ip_address = database[request_name]
            response = f"TYPE=A NAME={request_name}\nVALUE={ip_address}\nTTL=10"
            sock.sendto(response.encode(), addr)
        else:
            # Send a response with code 404
            response = "404 Not Found"
            sock.sendto(response.encode(), addr)


KeyboardInterrupt: 